In [28]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


data_json=[{"Gender": "Male", "HeightCm": 171, "WeightKg": 96 },
{ "Gender": "Male", "HeightCm": 161, "WeightKg": 85 },
{ "Gender": "Male", "HeightCm": 180, "WeightKg": 77 },
{ "Gender": "Female", "HeightCm": 166, "WeightKg": 62},
{"Gender": "Female", "HeightCm": 150, "WeightKg": 70},
{"Gender": "Female", "HeightCm": 167, "WeightKg": 82}]


data=pd.json_normalize(data_json)

# introducing error in one of the records
data.loc[1,'HeightCm']='incorrect value'

data.head()

,Gender,HeightCm,WeightKg
0,Male,171,96
1,Male,incorrect value,85
2,Male,180,77
3,Female,166,62
4,Female,150,70


In [29]:
bmi_table2={18.4:('Under Weight','Malnutrition risk')
            ,18.5:('Normal Weight','Low risk'),
            25:('Over Weight','Enhanced risk'),
            30:('Moderately Obese','Medium risk'),
            35:('Severely Obese','High risk'),
            40:('Very Severely Obese','Very high risk')}
bmi_table2

{18.4: ('Under Weight', 'Malnutrition risk'),
 18.5: ('Normal Weight', 'Low risk'),
 25: ('Over Weight', 'Enhanced risk'),
 30: ('Moderately Obese', 'Medium risk'),
 35: ('Severely Obese', 'High risk'),
 40: ('Very Severely Obese', 'Very high risk')}

In [30]:
def bmi_check(gender,h,w):
    # bmi calculated
    bmi_val = w/((h/100)**2)
    
    #fetch bmi category and bmi range for dictionary
    
    if bmi_val <= 18.4 :
        bmi_cat,bmi_range = bmi_table2[18.4]

    elif bmi_val >=18.5 and bmi_val <= 24.9:
        bmi_cat,bmi_range = bmi_table2[18.5]
        
    elif bmi_val >=25 and bmi_val <= 29.9:
        bmi_cat,bmi_range = bmi_table2[25]

    elif bmi_val >=30 and bmi_val <= 34.9:
        bmi_cat,bmi_range = bmi_table2[30]
        
    elif bmi_val >= 35 and bmi_val <= 39.9:
        bmi_cat,bmi_range = bmi_table2[35]
        
    elif bmi_val > 40:
        bmi_cat,bmi_range = bmi_table2[40]
    
    return bmi_cat,bmi_range



#records with some data issues

incorrect_records=data.loc[~(data['Gender'].astype(str).str.isalpha() & data['HeightCm'].astype(str).str.isdigit() & data['WeightKg'].astype(str).str.isdigit() )]
#fill values in case of faulty records
incorrect_records.loc[:,'bmi_cat']='Error with Records'
incorrect_records.loc[:,'bmi_range']='Error with Records'

#final records to check BMI 
data=data.loc[(data['Gender'].astype(str).str.isalpha() & data['HeightCm'].astype(str).str.isdigit() & data['WeightKg'].astype(str).str.isdigit() )]


for index,row in data.iterrows():
    #check data type should be correct
    bmi_cat,bmi_range=bmi_check(row['Gender'],row['HeightCm'],row['WeightKg'])
    data.loc[index,'bmi_cat']=bmi_cat
    data.loc[index,'bmi_range']=bmi_range

In [31]:
#incorrect records are placed at the end of the dataframe
pd.concat([data,incorrect_records],axis=0).reset_index(drop=True)

,Gender,HeightCm,WeightKg,bmi_cat,bmi_range
0,Male,171,96,Moderately Obese,Medium risk
1,Male,180,77,Normal Weight,Low risk
2,Female,166,62,Normal Weight,Low risk
3,Female,150,70,Moderately Obese,Medium risk
4,Female,167,82,Over Weight,Enhanced risk
5,Male,incorrect value,85,Error with Records,Error with Records


In [32]:
# for converting the dataframe into json
pd.concat([data,incorrect_records],axis=0).reset_index(drop=True).T.to_json()

'{"0":{"Gender":"Male","HeightCm":171,"WeightKg":96,"bmi_cat":"Moderately Obese","bmi_range":"Medium risk"},"1":{"Gender":"Male","HeightCm":180,"WeightKg":77,"bmi_cat":"Normal Weight","bmi_range":"Low risk"},"2":{"Gender":"Female","HeightCm":166,"WeightKg":62,"bmi_cat":"Normal Weight","bmi_range":"Low risk"},"3":{"Gender":"Female","HeightCm":150,"WeightKg":70,"bmi_cat":"Moderately Obese","bmi_range":"Medium risk"},"4":{"Gender":"Female","HeightCm":167,"WeightKg":82,"bmi_cat":"Over Weight","bmi_range":"Enhanced risk"},"5":{"Gender":"Male","HeightCm":"incorrect value","WeightKg":85,"bmi_cat":"Error with Records","bmi_range":"Error with Records"}}'